In [1]:
import os
os.environ.pop("http_proxy", None)
os.environ.pop("https_proxy", None)
os.environ["NO_PROXY"] = "localhost,127.0.0.1"

import sys
sys.path.append("..")

from config import Config
from unimemrag.retriever import ClipEmbedding
from unimemrag.memory_forest.memory_forest import MemoryForestStore

cfg = Config(collection="memtree")                          
embed_model = ClipEmbedding(model_name="../../ckpts/clip-vit-base-patch32")
memforest_store = MemoryForestStore(cfg, vector_size=embed_model.dim)

/hpc2hdd/home/sguo349/miniconda3/envs/unimemrag/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/hpc2hdd/home/sguo349/miniconda3/envs/unimemrag/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/hpc2hdd/home/sguo349/junjiejiang/algorithm/UniMemRAG/examples/../unimemrag/vector_store/qdrant.py:40: UserWarning: Qdrant client version 1.15.1 is incompatib

In [ ]:
import os
from openai import OpenAI
from sklearn import tree
os.environ.pop("http_proxy", None)
os.environ.pop("https_proxy", None)
os.environ["NO_PROXY"] = "localhost,127.0.0.1"
import json
from pathlib import Path
from dataclasses import asdict
from itertools import islice
from tqdm.auto import tqdm
# import sys
# sys.path.append("..")
from unimemrag.memory_forest.memory_forest import build_tree, iter_wiki_dict
from unimemrag.utils.image_cache import download_images_for_kb, load_image_cache, replace_payload_image_urls, save_image_cache

llm = OpenAI(
       api_key="sk-d6de5f2c6e4c458e9b53e9d96a5e75bb",
       base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

KB_PATH = Path("../../benchmark/infoseek/subset/wiki_text/wiki_5k_dict.json")

with KB_PATH.open("r", encoding="utf-8") as fh:
    KB = json.load(fh)

IMAGE_CACHE_DIR = Path('../../benchmark/infoseek/subset/wiki_text/images_5k')
IMAGE_CACHE_INDEX = IMAGE_CACHE_DIR / "image_cache_index.json"
image_cache = load_image_cache(IMAGE_CACHE_INDEX)

if not image_cache:
    image_cache = download_images_for_kb(KB, IMAGE_CACHE_DIR, max_workers=64, resume=True)
    save_image_cache(image_cache, IMAGE_CACHE_INDEX)
else:
    print(f"Loaded {len(image_cache)} cached entries from {IMAGE_CACHE_INDEX}")

def localize_payload(payload):
    return replace_payload_image_urls(dict(payload), image_cache)


total = len(KB)
iterator = islice(iter_wiki_dict(KB), total)
iterator = tqdm(iterator, total=total, desc="Building Trees")

trees = []
for wiki_url, payload in iterator:
    payload = localize_payload(payload)
    # tree = build_tree(wiki_url, payload)
    tree = build_tree(
       wiki_url,
       payload,
       llm=llm,
       llm_model="qwen3-8b",
       max_summary_sections=None,
       llm_workers=10,
       llm_request_kwargs={"max_tokens": 512, "temperature": 0.7, "extra_body": {"enable_thinking": False}}
       )      
    trees.append(tree)

TREES_OUT = Path("trees_5k.json")
with TREES_OUT.open("w", encoding="utf-8") as fh:
    json.dump([asdict(t) for t in trees], fh, indent=2)
print(f"Saved {len(trees)} trees to {TREES_OUT}")

In [2]:
'''
load trees from file
'''

import json
from pathlib import Path
from unimemrag.memory_forest.memory_forest import MemoryTree, RootNode, EventNode, LeafNode

TREES_OUT = Path("trees_5k.json")

with TREES_OUT.open("r", encoding="utf-8") as fh:
    data = json.load(fh)

def tree_from_dict(d):
    root = RootNode(**d["root"])
    events = [EventNode(**e) for e in d.get("events", [])]
    leaves = [LeafNode(**l) for l in d.get("leaves", [])]
    return MemoryTree(tree_id=d["tree_id"], root=root, events=events, leaves=leaves)

trees = [tree_from_dict(x) for x in data]

In [ ]:
# memforest_store.ingest_trees(
#       trees,
#       embed_model,
#       batch_size=256,
#       text_workers=16,
#       image_workers=16,
#       alpha=0.1,
#       show_progress=True
# )

# 入库：多模态 leaf
memforest_store.ingest_trees_new(
    trees,
    embed_model,
    beta=0.6,
    batch_size=256,
    text_workers=16,
    image_workers=16,
    show_progress=True,
)

Embedding Trees (fused leaves):  57%|█████▋    | 30694/53538 [00:30<00:09, 2423.77it/s]/hpc2hdd/home/sguo349/miniconda3/envs/unimemrag/lib/python3.11/site-packages/PIL/Image.py:3452: DecompressionBombWarning: Image size (154443312 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/hpc2hdd/home/sguo349/miniconda3/envs/unimemrag/lib/python3.11/site-packages/PIL/Image.py:3452: DecompressionBombWarning: Image size (150000000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/hpc2hdd/home/sguo349/miniconda3/envs/unimemrag/lib/python3.11/site-packages/PIL/Image.py:3452: DecompressionBombWarning: Image size (102345000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/hpc2hdd/home/sguo349/miniconda3/envs/unimemrag/lib/python3.11/site-packages/PIL/Image.py:3452: DecompressionBombWarning: Image size (89972928 pixels) exceeds limit of 89478485 pixels,

In [41]:
"""
Loading tree index from memforest store
"""

import sys
sys.path.append('..')

import os
os.environ.pop("http_proxy", None)
os.environ.pop("https_proxy", None)
os.environ["NO_PROXY"] = "localhost,127.0.0.1"

from config import Config
from unimemrag.embedding.models.ClipEmbedding import ClipEmbedding
from unimemrag.memory_forest import MemoryForestStore

cfg = Config(collection="memtree")
embed_model = ClipEmbedding()
memforest_store = MemoryForestStore(cfg, vector_size=embed_model.dim)

/hpc2hdd/home/sguo349/junjiejiang/algorithm/UniMemRAG/examples/../unimemrag/vector_store/qdrant.py:40: UserWarning: Qdrant client version 1.15.1 is incompatible with server version 1.12.0. Major versions should match and minor version difference must not exceed 1. Set check_compatibility=False to skip version check.
  self.client = QdrantClient(


In [ ]:
# results = memforest_store.retrieve(
#     embed_model,
#     query_text="What is the closest parent taxonomy of this bird?",
#     query_image="../../benchmark/oven/05/oven_05037337.JPEG",
#     root_top_k=3,
#     event_top_k=3,
#     leaf_top_k=3,
#     alpha=0.3,
# )

# 检索：collapsed retrieval
results = memforest_store.collapsed_retrieve(
    embed_model,
    alpha=0.1,
    query_text="where is this plant originally discovered?",
    query_image="../../benchmark/oven/05/oven_05031362.jpg",
    # query_image="https://projectupland.com/wp-content/uploads/2025/07/California-Quail-Callipepla-californica-The-Valley-Quail-Life-History.jpg",
    leaf_top_k=5,
)

In [37]:
# results[1].leaves
for result in results:
    print(result)

CollapsedRetrievalResult(tree_id='Q373530', events=[RetrievalHit(id='a0f5925c-6bca-5d38-8b04-504563b5908e', score=0.9019407, payload={'uri': 'Q373530::evt11', 'modality': 'text', 'node_type': 'event', 'tree_id': 'Q373530', 'parent_id': 'Q373530', 'summary': '==== Companion plant ====', 'leaf_ids': ['Q373530::evt11::leaf0'], 'metadata': {'paragraph_index': 11}}, role=<NodeRole.EVENT: 'event'>, collection='memtree_events')], leaves={'a0f5925c-6bca-5d38-8b04-504563b5908e': [RetrievalHit(id='a15d2437-d998-57ce-8543-692f603d4d9c', score=0.9019407, payload={'uri': 'Q373530::evt11::leaf0', 'modality': 'text', 'node_type': 'leaf', 'tree_id': 'Q373530', 'parent_id': 'Q373530::evt11', 'content': '==== Companion plant ====', 'metadata': {'paragraph_index': 11, 'chunk_index': 0}}, role=<NodeRole.LEAF: 'leaf'>, collection='memtree_leaves')]})
CollapsedRetrievalResult(tree_id='https://en.wikipedia.org/wiki/Chrysanthemum_%C3%97_morifolium', events=[RetrievalHit(id='e557854c-53f3-5672-9507-c804d58c5d8

In [ ]:
for tree in results:
    print("=== Root:", tree.root.payload.get("topic"))
    for event in tree.events:
        sec_id = event.id
        meta = event.payload.get("metadata", {})
        title = meta.get("section_title") or event.payload.get("summary")
        section_chunks = [
            leaf.payload["content"]
            for leaf in tree.leaves.get(sec_id, [])
            if "content" in leaf.payload
        ]
        section_text = "\n".join(section_chunks)
        print(f"\nSection: {title}")
        print("Images:", meta.get("section_images", []))
        print("Text:", section_text)

In [ ]:
# memforest_store.client.delete_collection(memforest_store.cfg.collection)
# memforest_store.client.delete_collection(memforest_store.event_collection)
# memforest_store.client.delete_collection(memforest_store.leaf_collection)

True

In [ ]:
memforest_store.client.count(memforest_store.roof_collection, exact=True)
# memforest_store.client.count(memforest_store.leaf_collection, exact=True)

In [ ]:
for res in results:
    print(res.root.payload["topic"])
    print(res.events)

In [ ]:
from unimemrag.vlm.QwenVL import QwenVL
vlm = QwenVL(
    model_path="../../ckpts/Qwen2.5-VL-7B-Instruct",
    torch_dtype="auto",
    device_map="auto",
    attn_implementation="flash_attention_2",
)

In [ ]:
result = results[0]

def format_context(tree_result, top_sections=3, max_chars=1024):
    root = tree_result.root
    meta = root.payload.get("metadata", {}) or {}
    lines = [
        f"Topic: {root.payload.get('topic') or meta.get('source_url', 'n/a')}",
        f"Tree ID: {tree_result.tree_id}",
        f"Alignment score: {meta.get('alignment_best_score', 'n/a')}",
        "",
    ]
    for event in tree_result.events[:top_sections]:
        emeta = event.payload.get("metadata", {}) or {}
        title = emeta.get("section_title") or event.payload.get("summary") or "Unknown section"
        lines.append(f"Section: {title}")
        section_preview = (emeta.get("section_preview") or event.payload.get("summary") or "").strip()
        if section_preview:
            lines.append(section_preview)
        leaf_hits = tree_result.leaves.get(event.id, [])
        for idx, leaf_hit in enumerate(leaf_hits[:2], start=1):
            snippet = (leaf_hit.payload.get("content") or "").strip()
            if not snippet:
                continue
            lines.append(f"Paragraph {idx}: {snippet}")
        lines.append("")
    context = "\n".join(lines).strip()
    if max_chars and max_chars > 0 and len(context) > max_chars:
        truncated = context[:max_chars]
        if "\n" in truncated:
            truncated = truncated.rsplit("\n", 1)[0]
        context = truncated
    return context

context = format_context(result, top_sections=3)
print("formated_context:", context)
question="Who built this object that has not cash dispensing features?",

messages = [
    {"role": "system", "content": [
        {"type": "text", "text": "You are a helpful assistant. Please answer the question based on the context provided."}
    ]},
    {"role": "user", "content": [
        {"type": "image", "image": "../atm-machine.jpg"},
        {"type": "text",  "text": f"Here's the contexts:\n{context}\n\nNow, answer the question:\n{question}"}
    ]}
]
print(messages)

answer = vlm.chat(messages, max_new_tokens=32768, temperature=0.7)
print(answer)

In [ ]:
import json

with open("../../benchmark/infoseek/wiki_text/wiki_100_dict_v4.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [ ]:
for i, (k, v) in enumerate(data.items()):
    print(f"[{i}] {k} -> {list(v.keys())}")
    print(f"[{i}] {k} -> topic: {v.get('title', [])} image_urls: {v.get('image_urls', [])}")
    if i >= 5:
        break

In [ ]:
import os
os.environ.pop("http_proxy", None)
os.environ.pop("https_proxy", None)
os.environ["NO_PROXY"] = "localhost,127.0.0.1"

import sys
sys.path.append('..')

from unimemrag.embedding.models.ClipEmbedding import ClipEmbedding
import numpy as np

clip = ClipEmbedding()

In [ ]:
import os
os.environ.pop("http_proxy", None)
os.environ.pop("https_proxy", None)
os.environ["NO_PROXY"] = "localhost,127.0.0.1"

import sys
sys.path.append("..")
from unimemrag.retriever import ClipEmbedding

embed_model = ClipEmbedding(model_name="../../ckpts/clip-vit-base-patch32")

topic = "Reflecting telescope"
image_urls = [
    "https://upload.wikimedia.org/wikipedia/commons/c/c1/ParasolMushroom.JPG",
    "https://upload.wikimedia.org/wikipedia/commons/8/8c/Parasol-1.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/5/54/Glawlen_y_Bwgan_%28Macrolepiota_procera%29.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/a/a4/Macrolepiota-procera.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/d/d3/Macrolepiota_procera_fungus%2C_Woodfidley%2C_New_Forest_-_geograph.org.uk_-_261237.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/4/48/Macrolepiota_procera_2011_G1.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/c/cf/Parasol-Macrolepiota-procera.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/6/6a/Macrolepiota_procera_2013_G1.jpg",
    "http://upload.wikimedia.org/wikipedia/commons/a/ab/Parasol_mushroom.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/d/da/Breaded_parasol_mushroom.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/1/1b/Edible_fungi_in_basket_2019_G2.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/f/f3/Unopened_parasol_mushroom.jpg"
    ]


text_embed = embed_model.embed_texts([topic])[0]      # (dim,)
image_embeds = embed_model.embed_images(image_urls)   # (N, dim)

scores = image_embeds @ text_embed             # 余弦相似度
for url, score in zip(image_urls, scores):
    print(f"{url} -> {score:.4f}")